In [14]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage
import tiktoken

In [15]:
examples = [
    {"input": "Gollum", "output": "<Story involving Gollum>"},
    {"input": "Gandalf", "output": "<Story involving Gandalf>"},
    {"input": "Bilbo", "output": "<Story involving Bilbo>"},
]

In [16]:
story_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Character: {input}\nStory: {output}",
)

In [17]:
def num_tokens_from_string(string: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding("cl100k_base")
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [18]:
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=story_prompt,
    max_length=1000, # 1000 tokens are to be included from examples in the prompt
    # get_text_length: Callable[[str], int] = lambda x: len(re.split("\n| ", x))
    # You have modified the get_text_length function to work with the TikToken library based on token usage:
    get_text_length=num_tokens_from_string,
)

In [19]:
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=story_prompt,
    prefix="Generate a story for {character} using the current Character/Story pairs from all of the characters as context.",
    suffix="Character: {character}\nStory:",
    input_variables=["character"],
)

In [20]:
# Provide a new character from lord of the rings
formatted_prompt = dynamic_prompt.format(character="Frodo")

In [23]:
print(formatted_prompt)

Generate a story for Frodo using the current Character/Story pairs from all of the characters as context.

Character: Gollum
Story: <Story involving Gollum>

Character: Gandalf
Story: <Story involving Gandalf>

Character: Bilbo
Story: <Story involving Bilbo>

Character: Frodo
Story:


In [29]:
# Using the traditional way of using the model with __call__:
model = ChatOpenAI()

result = model.invoke([SystemMessage(content=formatted_prompt)])

print(result.content)

Once upon a time in the peaceful land of Middle-earth, there lived a young hobbit named Frodo Baggins. Frodo had always admired his uncle, Bilbo Baggins, who had gone on an incredible adventure and returned with tales of the One Ring. Little did Frodo know that he would soon follow in his uncle's footsteps and embark on a journey that would change his life forever.

One day, Frodo received a visit from the wise wizard Gandalf. The old wizard revealed to Frodo the truth about the One Ring, a ring of immense power that had the ability to corrupt those who possessed it. Gandalf told Frodo that the ring had to be destroyed in the fiery depths of Mount Doom, where it was forged, to prevent it from falling into the wrong hands.

Frodo, filled with determination and a sense of duty, volunteered to be the ring-bearer and take on the perilous quest to destroy the One Ring. With the guidance of Gandalf and the support of his loyal friends, Samwise Gamgee, Merry, and Pippin, Frodo set off on his 